In [88]:
import numpy as np
import pandas as pd
import matplotlib
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import os
import pickle

Create numpy array of embeddings

In [2]:
embedding_list = []

In [3]:
timestamps = []

In [4]:
for folder in \
    os.listdir('/green-projects/project-sonyc_redhook/workspace/share/redhook-analysis/output/embeddings'):
        for file in os.listdir\
        ('/green-projects/project-sonyc_redhook/workspace/share/redhook-analysis/output/embeddings/' + folder):
            timestamps.append(file.split('.')[0])
            data = \
            np.load('/green-projects/project-sonyc_redhook/workspace/share/redhook-analysis/output/embeddings/'\
                    + folder + '/' + file)
            emb = data['embedding']
            embedding_list.append(emb)
            data.close()

In [5]:
len(embedding_list)

1523

In [6]:
embedding_list = np.asarray(embedding_list)

In [7]:
embedding_list.shape

(1523, 19, 512)

In [8]:
expanded_embedding_list = []

In [9]:
for embedding in embedding_list:
    for i in embedding:
        expanded_embedding_list.append(i)

In [10]:
len(expanded_embedding_list)

28937

In [11]:
expanded_embedding_list = np.asarray(expanded_embedding_list)

In [12]:
expanded_embedding_list.shape

(28937, 512)

In [13]:
pca_45 = sklearnPCA(45)
pca_45.fit(expanded_embedding_list)

PCA(copy=True, iterated_power='auto', n_components=45, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [14]:
embedding_list = np.asarray([pca_45.transform(embedding) for embedding in embedding_list])

In [15]:
embedding_list.shape

(1523, 19, 45)

In [16]:
def get_summary(embedding):
    embedding_mean = np.mean(embedding, axis=0)
    embedding_std = np.std(embedding, axis=0)
    embedding_max = np.amax(embedding, axis=0)
    embedding_min = np.amin(embedding, axis=0)
    embedding_summary = np.concatenate((embedding_mean, embedding_max, embedding_min, embedding_std))
    return embedding_summary

In [17]:
embedding_summaries = np.asarray([get_summary(embedding) for embedding in embedding_list])

In [18]:
embedding_summaries.shape

(1523, 180)

Joining embeddings with timestamp label

In [20]:
timestamps = np.asarray(timestamps)

In [32]:
timestamps.shape

(1523,)

In [63]:
embedding_summaries.shape

(1523, 180)

In [73]:
labeled_embedding_summaries = []

In [74]:
for i in range(len(timestamps)):
    labeled_embedding_summaries.append([timestamps[i], embedding_summaries[i]])

In [76]:
labeled_embedding_summaries = np.asarray(labeled_embedding_summaries)

In [80]:
labeled_embedding_summaries.shape

(1523, 2)

In [89]:
labeled_embedding_summaries[0][1].shape

(180,)

Random Forest on 

Creating Y list of classifications

In [81]:
with open('/green-projects/project-sonyc_redhook/workspace/share/redhook-analysis/output/annotation_list.pickle', "rb") as f:
       annotation_list = pickle.load(f)

In [82]:
len(annotation_list)

1538